Nandhini


Week 9 Machine Learning Assignment

Among the different classification models included in the Python notebook, which model had the best overall performance? Support your response by referencing appropriate evidence.

To guarantee model performance on new data we must evaluate its accuracy on training data and test data. The results table showed multiple evaluation of logistic regression models with different penalties and random forest classification methods.
The standard Logistic Regression model and the Logistic Regression with L1 penalty (C=10) demonstrated the best generalization ability since they reached a test accuracy of 0.718. The training accuracy of Logistic Regression with L1 penalty and C=10 reached 0.7347 while maintaining 0.7333 test accuracy which indicated better pattern recognition without overfitting the data.
The training accuracy score of 0.9993 was produced by the Random Forest model without cross-validation yet its test results revealed a poor accuracy rate of 0.686 because it had memorized training data without achieving effective generalization. Models utilizing cross-validation demonstrated better equilibrium between the performance on training and test data points.
The Logistic Regression with L1 penalty and C=10 (Logistic_L1_C_10) emerges as the most suitable model based on the evaluation results. Logistic_L1_C_10 represents a model that achieves the right combination of complexity and generalization ability which delivers consistent performance between training and test data while avoiding overfitting like more complex models do.

Next, fit a series of logistic regression models, without regularization. Each model should use the same set of predictors (all of the relevant predictors in the dataset) and should use the entire dataset, rather than a fraction of it. Use a randomly chosen 80% proportion of observations for training and the remaining for checking the generalizable performance (i.e., performance on the holdout subset). Be sure to ensure that the training and holdout subsets are identical across all models. Each model should choose a different solver.



In [ ]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from patsy import dmatrices

In [ ]:
patient = pd.read_csv('./PatientAnalyticFile.csv')

# Create mortality variable
patient['mortality'] = np.where(patient['DateOfDeath'].isnull(), 0, 1)

# Convert DateOfBirth to date and calculate age
patient['DateOfBirth'] = pd.to_datetime(patient['DateOfBirth'])
patient['Age_years'] = ((pd.to_datetime('2015-01-01') - patient['DateOfBirth']).dt.days/365.25)

# Create formula for all variables in model
vars_remove = ['PatientID', 'First_Appointment_Date', 'DateOfBirth',
               'Last_Appointment_Date', 'DateOfDeath', 'mortality']
vars_left = set(patient.columns) - set(vars_remove)
formula = "mortality ~ " + " + ".join(vars_left)

In [ ]:
Y, X = dmatrices(formula, patient)
y = np.ravel(Y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']

In [ ]:
results_list = []
for solver in solvers:
    start_time = time.time()

    # Handle special case for liblinear
    if solver == 'liblinear':
        model = LogisticRegression(solver=solver, penalty='l2', C=1e5, random_state=42, max_iter=1000)
    else:
        model = LogisticRegression(solver=solver, penalty=None, random_state=42, max_iter=1000)

    model.fit(X_train, y_train)

    time_taken = time.time() - start_time

    y_train_pred = model.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)

    y_test_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    results_list.append({
        'Solver used': solver,
        'Training subset accuracy': round(train_accuracy, 4),
        'Holdout subset accuracy': round(test_accuracy, 4),
        'Time taken': round(time_taken, 4)
    })

results_df = pd.DataFrame(results_list)
print(results_df)

  Solver used  Training subset accuracy  Holdout subset accuracy  Time taken
0   newton-cg                    0.7481                   0.7355      0.3283
1       lbfgs                    0.7479                   0.7355      0.3247
2   liblinear                    0.7479                   0.7362      0.0570
3         sag                    0.7479                   0.7358      4.0442
4        saga                    0.7480                   0.7360      4.5576


Based on the results, which solver yielded the best results? Explain the basis for ranking the models - did you use training subset accuracy? Holdout subset accuracy? Time of execution? All three? Some combination of the three?


The five solvers demonstrated equivalent training subset accuracy performance through values between 0.7479 and 0.7481. The accuracy values between all solvers during training differ by only 0.0002 points which indicates equivalent training results. The solvers demonstrated a high degree of similarity when they solved this specific dataset because they reached nearly equivalent solutions.

Holdout subset accuracy results span from 0.7355 to 0.7360 with a minor difference between the highest and lowest values. Among the solvers tested, the 'saga' achieved the peak holdout accuracy score of 0.7360 followed by 'liblinear' at 0.7362 and then by 'sag' at 0.7358. The 'newton-cg' and 'lbfgs' solvers reached the same minimum value of 0.7355. A difference of 0.0005 exists between the highest and lowest accuracy scores although this amount may prove important when working with large datasets or critical applications.

Execution time reveals the greatest distinctions between the different solvers. The 'liblinear' solver outpaced all other solvers by completing the task in only 0.0570 seconds which corresponded to 5-6 times faster than newton-cg and lbfgs and 70-80 times faster than sag and saga solvers. The execution time of 'sag' and 'saga' exceeded 4 seconds each.

The 'liblinear' solver demonstrates superior performance across every aspect when solving this particular problem. The 'liblinear' solver delivered the second-best holdout accuracy performance at the same time it operated as the most efficient computational method. Within applications demanding fast repeated model training and handling big datasets the speed benefit of the liblinear solver would stand out the most. The significant difference in execution time between saga (80 times longer than liblinear) escalates difficulty of justifying its minimal holdout accuracy improvement of 0.0002.